<a href="https://colab.research.google.com/github/Varsha17112005/sample/blob/main/catathon_rtmde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision timm opencv-python gradio ultralytics matplotlib --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
import cv2
from PIL import Image
import gradio as gr
from ultralytics import YOLO
import timm


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
class RTMDE_MobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('mobilenetv3_small_050', pretrained=True, features_only=True)
        last_channels = self.backbone.feature_info[-1]['num_chs']
        self.decoder = nn.Sequential(
            nn.Conv2d(last_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 1, kernel_size=1)
        )

    def forward(self, x):
        features = self.backbone(x)[-1]
        return self.decoder(features)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RTMDE_MobileNet().to(device)
model.eval()
yolo_model = YOLO("yolov8n.pt")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

100%|██████████| 6.25M/6.25M [00:00<00:00, 65.8MB/s]


In [ ]:
def estimate_depth(input_image):
    image = Image.fromarray(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    input_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        depth_map = model(input_tensor).squeeze().cpu().numpy()

    depth_map = cv2.resize(depth_map, (input_image.shape[1], input_image.shape[0]))
    depth_map = (depth_map - np.min(depth_map)) / (np.max(depth_map) - np.min(depth_map) + 1e-6)
    heat_map = cv2.applyColorMap((depth_map * 255).astype(np.uint8), cv2.COLORMAP_JET)
    return depth_map, heat_map


In [ ]:
def detect_objects(input_image, depth_map):
    results = yolo_model(input_image)
    detected_image = input_image.copy()
    pov_x, pov_y = detected_image.shape[1] // 2, detected_image.shape[0] // 2
    distance_results = []

    label_counter = {}

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cls_id = int(box.cls[0])
            label = result.names[cls_id]
            label_counter[label] = label_counter.get(label, 0) + 1
            label_id = f"{label} {label_counter[label]}"

            object_depth = np.mean(depth_map[y1:y2, x1:x2])
            if np.isnan(object_depth) or object_depth <= 0:
                estimated_distance = "N/A"
            else:
                estimated_distance = round(15 / object_depth, 2)

            obj_cx, obj_cy = (x1 + x2) // 2, (y1 + y2) // 2
            pov_dist = round(np.sqrt((pov_x - obj_cx)**2 + (pov_y - obj_cy)**2), 2)

            cv2.rectangle(detected_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(detected_image, label_id, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

            distance_results.append(f"{label_id}: {estimated_distance}m | POV Dist: {pov_dist}px")

    return detected_image, "\n".join(distance_results)


In [ ]:
def process_image(input_image):
    if input_image is None:
        return None, None, "No input image."
    depth_map, heat_map = estimate_depth(input_image)
    detected_image, distance_text = detect_objects(input_image, depth_map)
    return detected_image, heat_map, distance_text


In [ ]:
iface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="numpy", label="Upload or Capture Image"),
    outputs=[
        gr.Image(type="numpy", label="Object Detection + Labels"),
        gr.Image(type="numpy", label="Depth Heatmap"),
        gr.Textbox(label="Distances from Camera POV")
    ],
    live=True
)

iface.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://93ebf4096306f3b763.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 448x640 10 cars, 3 buss, 1 truck, 188.9ms
Speed: 11.2ms preprocess, 188.9ms inference, 27.3ms postprocess per image at shape (1, 3, 448, 640)
